# Jacobi Example

In [ ]:
# dependencies
using LFAToolkit
using LinearAlgebra
using Plots

In [ ]:
# setup
dimension = 2
p = 2
mesh = []
if dimension == 1
   mesh = Mesh1D(1.0)
elseif dimension == 2
   mesh = Mesh2D(1.0, 1.0)
end
basis = TensorH1LagrangeBasis(p, p+1, dimension)

function diffusionweakform(du::Array{Float64}, w::Array{Float64})
    dv = du*w[1]
    return [dv]
end

# diffusion operator
inputs = [
    OperatorField(basis, [EvaluationMode.gradient]),
    OperatorField(basis, [EvaluationMode.quadratureweights]),
]
outputs = [OperatorField(basis, [EvaluationMode.gradient])]
diffusion = Operator(diffusionweakform, mesh, inputs, outputs)

# Jacobi smoother
jacobi = Jacobi(diffusion)

In [ ]:
# compute operator symbols
numberruns = 100

# compute and plot smoothing factor
if dimension == 1
    # compute
    maxeigenvalues = zeros(numberruns)
    for i in 1:numberruns
        A = computesymbols(jacobi, [0.838], [2π*(i/numberruns)])
        eigenvalues = [abs(val) for val in eigvals(A)]
        maxeigenvalues[i] = max(eigenvalues...)
    end

    # plot
    plot(maxeigenvalues, title="Jacobi Symbols")
elseif dimension == 2
    # compute
    maxeigenvalues = zeros(numberruns, numberruns)
    for i in 1:numberruns, j in 1:numberruns
        A = computesymbols(jacobi, [0.838], [2π*(i/numberruns), 2π*(j/numberruns)])
        eigenvalues = [abs(val) for val in eigvals(A)]
        maxeigenvalues[i, j] = max(eigenvalues...)
    end

    # plot
    heatmap(maxeigenvalues, title="Jacobi Symbols", transpose=true)
end